In [1]:
!cp -r /kaggle/input/fish4-5/stockfish-dd-src/src_c11 /kaggle/working/
!sudo apt-get update --quiet
!sudo apt-get install -y g++ make --quiet

Get:1 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:5 https://packages.cloud.google.com/apt cloud-sdk InRelease [1618 B]
Get:6 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [30.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:8 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3482 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [4227 kB]
Get:10 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1600 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4639 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1296 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security

In [2]:
%cd /kaggle/working/src_c11
!make build ARCH=x86-64

/kaggle/working/src_c11
make ARCH=x86-64 COMP=gcc config-sanity
make[1]: Entering directory '/kaggle/working/src_c11'

Config:
debug: 'no'
optimize: 'yes'
arch: 'x86_64'
os: 'any'
bits: '64'
prefetch: 'yes'
bsfq: 'yes'
popcnt: 'no'
sse: 'yes'

Flags:
CXX: g++
CXXFLAGS: -Wall -Wcast-qual -fno-exceptions -fno-rtti -std=c++11  -pedantic -Wno-long-long -Wextra -Wshadow -DNDEBUG -O3 -DIS_64BIT -msse -DUSE_BSFQ -flto
LDFLAGS:  -lpthread -Wall -Wcast-qual -fno-exceptions -fno-rtti -std=c++11  -pedantic -Wno-long-long -Wextra -Wshadow -DNDEBUG -O3 -DIS_64BIT -msse -DUSE_BSFQ -flto

Testing config sanity. If this fails, try 'make help' ...

make[1]: Leaving directory '/kaggle/working/src_c11'
make ARCH=x86-64 COMP=gcc all
make[1]: Entering directory '/kaggle/working/src_c11'
g++ -Wall -Wcast-qual -fno-exceptions -fno-rtti -std=c++11  -pedantic -Wno-long-long -Wextra -Wshadow -DNDEBUG -O3 -DIS_64BIT -msse -DUSE_BSFQ -flto   -c -o benchmark.o benchmark.cpp
g++ -Wall -Wcast-qual -fno-exceptions -f

In [3]:
!./stockfish uci

Stockfish DD 64 by Tord Romstad, Marco Costalba and Joona Kiiski
id name Stockfish DD 64
id author Tord Romstad, Marco Costalba and Joona Kiiski

option name Write Debug Log type check default false
option name Write Search Log type check default false
option name Search Log Filename type string default SearchLog.txt
option name Book File type string default book.bin
option name Best Book Move type check default false
option name Contempt Factor type spin default 0 min -50 max 50
option name Mobility (Midgame) type spin default 100 min 0 max 200
option name Mobility (Endgame) type spin default 100 min 0 max 200
option name Pawn Structure (Midgame) type spin default 100 min 0 max 200
option name Pawn Structure (Endgame) type spin default 100 min 0 max 200
option name Passed Pawns (Midgame) type spin default 100 min 0 max 200
option name Passed Pawns (Endgame) type spin default 100 min 0 max 200
option name Space type spin default 100 min 0 max 200
option name Aggressiveness type spin de

In [4]:
!mv stockfish /kaggle/working/
!strip /kaggle/working/stockfish

In [5]:
%%writefile /kaggle/working/main.py

import subprocess
import os
import psutil


class ChessEngine:
    def __init__(self, engine_path):
        self.engine = subprocess.Popen(
            [engine_path],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        print("RAM usage before engine initialization:", self.get_memory_usage(), "MB")
        self._initialize_engine()
        print("RAM usage after engine initialization:", self.get_memory_usage(), "MB")

    def _initialize_engine(self):
        self._send_command("uci")
        while True:
            output = self._read_output()
            if output == "uciok":
                break
        self._send_command("setoption name Hash value 1")

        
    def _send_command(self, command):
        self.engine.stdin.write(command + "\n")
        self.engine.stdin.flush()

    def _read_output(self):
        output = self.engine.stdout.readline().strip()
        return output

    def get_memory_usage(self):
        process = psutil.Process(self.engine.pid)
        return process.memory_info().rss / (1024 * 1024)  # Convert bytes to MB

    def get_best_move(self, fen, movetime=200):
        # print("RAM usage before thinking:", self.get_memory_usage(), "MB")
        self._send_command(f"position fen {fen}")
        self._send_command(f"go movetime {movetime}")
        best_move = None
        while True:
            output = self._read_output()
            if output.startswith("bestmove"):
                best_move = output.split()[1]
                break
        # print("RAM usage after thinking:", self.get_memory_usage(), "MB")
        print("")
        self._send_command("setoption name Clear Hash")
        return best_move

    def stop(self):
        self._send_command("quit")
        self.engine.terminate()
        self.engine.wait()

ultima = None

def chess_bot(obs):
    global ultima
    fen = obs['board']
    if os.path.exists('/kaggle_simulations/agent/stockfish'):
        engine_path = '/kaggle_simulations/agent/stockfish'
    else:
        engine_path = '/kaggle/working/stockfish'
    if ultima is None:
        ultima = ChessEngine(engine_path)
    best_move = ultima.get_best_move(fen)
    return best_move

Writing /kaggle/working/main.py


In [6]:
%cd /kaggle/working
!tar -czvf submission.tar.gz main.py mcu-max-uci

/kaggle/working
main.py
tar: mcu-max-uci: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


In [7]:
# !pip install --upgrade kaggle-environments --quiet
# from kaggle_environments import make
# env = make("chess", debug=True)

In [8]:
# result = env.run(["/kaggle/working/main.py","random"])
# for agent in result[-1]:
#    print("Status:", agent.status, "/ Reward:", agent.reward, "/ Time left:", agent.observation.remainingOverageTime)
# env.render(mode="ipython", width=700, height=700)